In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import glob
import os
import gc
import dask.dataframe as dd 


#book_snapshot_5
#liquidations
#quotes
#trades
#derivative_ticker

# path for all related files
datasets = r"C:\Users\danie\Desktop\master_project-env\datasets"
exchanges = ['ftx_']#, 'binance-futures_','ftx_', 'bybit_']
filetypes = ['book_snapshot_5','trades', 'quotes','derivative_ticker', 'liquidations']

In [2]:
path = r"C:\Users\danie\Desktop\master_project-env\datasets"
exchange = 'ftx'
for filetype in tqdm(filetypes):
    filename = glob.iglob(filetype+'*csv.gz')
    files_path = glob.glob(path + '\\' + exchange + '_' + filetype + '_' + "*.csv.gz") #file path for all files

    files = []

    for filename in files_path:
        df = pd.read_csv(filename, index_col=None, header=0)
        files.append(df)
    
    #prepare combined file
    combined = pd.concat(files, axis=0, ignore_index=True)
    combined['timestamp'] = pd.to_datetime(combined['timestamp'], unit='us')
    combined = combined.drop(['local_timestamp'], axis=1)
    combined = combined.set_index('timestamp').sort_index()
    combined = combined.dropna()
    
    combined.to_csv(path.replace('datasets', 'datasets_combined') + "\\" + exchange + "_"+ filetype 
                        + ".csv.gz", compression='gzip')

In [3]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 62889 entries, 2021-11-19 00:04:46.140152 to 2021-12-18 23:38:01.914502
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   exchange  62889 non-null  object 
 1   symbol    62889 non-null  object 
 2   id        62889 non-null  int64  
 3   side      62889 non-null  object 
 4   price     62889 non-null  int64  
 5   amount    62889 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 3.4+ MB


In [35]:
combined.memory_usage(deep=True)

Index        503112
exchange    3773340
symbol      4087785
id           503112
side        3813753
price        503112
amount       503112
dtype: int64

# Downsampling dataset to S

In [20]:
def downsampling(exchange, file_type):
    path = r"C:\Users\danie\Desktop\master_project-env\datasets_combined"
    # data[['exchange', 'symbol']] = data[['exchange', 'symbol']].apply(lambda x: x.astype('category'))
    
    #read file in chunks
    chunksize = 10**6
    combined_df = pd.DataFrame()
    for chunk in tqdm(pd.read_csv(path + "\\" + exchange + "_" + filetype +".csv.gz", chunksize=chunksize)):

        chunk['timestamp'] = pd.to_datetime(chunk['timestamp'])
        chunk = chunk.dropna()
        chunk = chunk.set_index('timestamp')
        chunk = chunk.resample('S').mean() #resample to S
        combined_df = pd.concat([combined_df, chunk])

    #find duplicate time indexes
    duplicates = [index for index, value in enumerate(combined_df.index.duplicated(keep='first')) if value]

    #replace second duplicate with mean for the duplicate pair
    combined_df.reset_index(inplace=True)
    for duplicate in tqdm(duplicates):
        new_average = combined_df[duplicate-1: duplicate+1].mean()
        combined_df = combined_df.replace(combined_df.iloc[duplicate], new_average)

    duplicates_new = [duplicate-1 for duplicate in duplicates] #index for the other duplicate pairs
    combined_df.drop(duplicates_new, inplace=True) #remove the duplicates
    combined_df  = combined_df.set_index('timestamp')
    
    #forward fill data that's missing after downsampling
    combined_df = combined_df.fillna(method='ffill')
    combined_df.to_parquet(path + "\\downsampled\\" + exchange + "_" + filetype + "_downsampled.parquet.gz", compression='gzip')
    
    return print("done {}".format(file_type))

In [21]:
exchange = 'ftx'
for filetype in filetypes:
    downsampling(exchange, filetype)
    

43it [01:54,  2.65s/it]
  0%|                                                                                           | 0/40 [00:00<?, ?it/s]C:\Users\danie\AppData\Local\Temp\ipykernel_2736\249732487.py:22: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  new_average = combined_df[duplicate-1: duplicate+1].mean()
100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:15<00:00,  2.61it/s]


done book_snapshot_5


15it [00:23,  1.53s/it]
  0%|                                                                                           | 0/12 [00:00<?, ?it/s]C:\Users\danie\AppData\Local\Temp\ipykernel_2736\249732487.py:22: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  new_average = combined_df[duplicate-1: duplicate+1].mean()
100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 14.62it/s]


done trades


32it [00:52,  1.64s/it]
  0%|                                                                                           | 0/27 [00:00<?, ?it/s]C:\Users\danie\AppData\Local\Temp\ipykernel_2736\249732487.py:22: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  new_average = combined_df[duplicate-1: duplicate+1].mean()
100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:02<00:00, 11.82it/s]


done quotes


1it [00:00, 333.04it/s]
0it [00:00, ?it/s]


done derivative_ticker


1it [00:00,  4.97it/s]
0it [00:00, ?it/s]


done liquidations
